### Closure Applications (Part 1)

В этом примере мы собираемся построить функцию усреднения, которая может усреднять несколько значений.

Суть в том, что мы просто хотим иметь возможность передавать числа в эту функцию и получать скользящее среднее значение с течением времени, а не усреднять список, что требует выполнения одних и тех же вычислений (суммирования и подсчета) снова и снова.

In [1]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [2]:
a = Averager()

In [3]:
a.add(10)

10.0

In [4]:
a.add(20)

15.0

In [5]:
a.add(30)

20.0

Мы можем сделать это, используя замыкание следующим образом:

In [6]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [7]:
a = averager()

In [8]:
a(10)

10.0

In [9]:
a(20)

15.0

In [10]:
a(30)

20.0

Теперь вместо того, чтобы хранить список и пересчитывать `total` и `count` каждый раз, когда нам понадобится новое среднее значение, мы будем хранить текущий итог и подсчет и обновлять каждое значение каждый раз, когда к текущему среднему значению добавляется новое значение, а затем возвращать `total / count`.

Давайте начнем с подхода класса, где мы будем использовать переменные экземпляра для хранения текущего итога и подсчета и предоставим метод экземпляра для добавления нового числа и возврата текущего среднего значения.

In [11]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0

    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [12]:
a = Averager()

In [13]:
a.add(10)

10.0

In [14]:
a.add(20)

15.0

In [15]:
a.add(30)

20.0

Теперь давайте посмотрим, как можно использовать замыкание, чтобы добиться того же самого.

In [16]:
def averager():
    total = 0
    count = 0

    def add(value):
        nonlocal total, count
        total += value
        count += 1
        return 0 if count == 0 else total / count

    return add


In [17]:
a = averager()

In [18]:
a(10)

10.0

In [19]:
a(20)

15.0

In [20]:
a(30)

20.0

#### Обобщая этот пример

Мы увидели, что нам по сути удалось преобразовать класс в эквивалентную функциональность с помощью замыканий. Это на самом деле верно в гораздо более общем смысле — очень часто классы, которые определяют один метод (кроме инициализаторов), могут быть реализованы с помощью замыкания.

Давайте рассмотрим другой пример этого.

Предположим, нам нужно что-то, что может отслеживать прошедшее время в секундах.

In [21]:
from time import perf_counter

In [22]:
class Timer:
    def __init__(self):
        self._start = perf_counter()

    def __call__(self):
        return (perf_counter() - self._start)

In [23]:
a = Timer()

Теперь подождите немного, прежде чем запускать следующую строку кода:

In [24]:
a()

0.011695334544051804

Давайте запустим еще один «таймер»:

In [25]:
b = Timer()

In [26]:
print(a())
print(b())

0.03528294403966765
0.011656054820407689


Теперь давайте перепишем это, используя замыкание:

In [27]:
def timer():
    start = perf_counter()

    def elapsed():
        # нам даже не нужно делать start nonlocal
        # так как мы только читаем
        return perf_counter() - start

    return elapsed

In [28]:
x = timer()

In [29]:
x()

0.011068213438975016

In [30]:
y = timer()

In [31]:
print(x())
print(y())

0.03419096772236116
0.01164738619174141


In [32]:
print(a())
print(b())
print(x())
print(y())

0.10822159832175349
0.08475345336494494
0.0462381944113351
0.023573252079387305


---